In [9]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import cv2
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
import datetime
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [10]:
data_dir = "64"

In [11]:
train_datagen = ImageDataGenerator(
    validation_split=0.1,
    rescale=1./255
)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    class_mode='categorical',
    target_size=(64, 64),
    subset='training',
    shuffle=True,
    batch_size=256,
    seed=0
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    class_mode='categorical',
    target_size=(64, 64),
    subset='validation',
    shuffle=True,
    batch_size=256,
    seed=0
)

Found 720000 images belonging to 200 classes.
Found 80000 images belonging to 200 classes.


In [14]:
def create_cnn_model(inShape, convNum, hNum, hAct, outNum, loss, drop_rate):
    """
    :param inShape: kształt wejścia, h*w*c
    :param convNum: lista ilości featerów w warstwach konwolucyjnych
    :param hNum: lista ilości neuronów w poszczególnych warstwach ukrytych
    :param hAct: rodzaj aktywacji w warstwach ukrytych, str
    :param outNum: ilość
    :param loss: funkcja kosztu
    """
    # pusty model
    model = tf.keras.Sequential()
    # warstwa wejściowa
    model.add(layers.InputLayer(input_shape=(inShape)))
    # warstwy konwolucyjne
    for idx, oneConvNum in enumerate(convNum):
        # dodajemy kolejną warstwę konwolucyjną
        # zmienia nam ilość cech(kanałów) na oneConvNum
        model.add(layers.Conv2D(
                                    filters=oneConvNum,
                                    kernel_size=3,
                                    strides=1,
                                    padding="SAME",
                                    activation="relu"
                                    )
                                )
        # doajemy następującego po convie maxpool'a
        # zmniejsza nam wymiarowość HxW o połowę
        model.add(layers.MaxPool2D(
                                   pool_size=(2, 2),
                                    strides=2,
                                   padding="SAME"
                                    )
                                )
        #model.add(layers.Dropout(drop_rate))
    # spłaszczamy po konwolucjach, przygotowanie pod relu
    model.add(layers.Flatten())
    # warstwy fully connected
    for idx, oneHidNum in enumerate(hNum):
        # dodajemy warstwę
        model.add(layers.Dense(oneHidNum, activation=hAct))
        model.add(layers.Dropout(drop_rate))
    # warstwa wyjściowa
    model.add(layers.Dense(outNum))
    # warstwa softmax, dystrybucja prawdopodobieństwa
    model.add(layers.Softmax())
    print(model.summary())
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.0015)
    
    model.compile(
        optimizer=opt,
        loss=loss,
        metrics=["accuracy"]
    )
    
    return model

In [16]:
test_generator = train_datagen.flow_from_directory(
    data_dir,
    class_mode='categorical',
    target_size=(64, 64),
    subset='validation',
    shuffle=True,
    batch_size=50000,
    seed=0
)

Found 80000 images belonging to 200 classes.


In [17]:
x_test, y_test = next(test_generator)

In [25]:
modelCNN3 = create_cnn_model(
    inShape=(64,64,3),
    convNum=[32,64,128],
    hNum=[256, 256, 256],
    hAct="relu",
    outNum=200,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    drop_rate = 0
)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 8, 8, 128)       

In [26]:
# dodajemy zapisywanie logów do tensorboard'a
log_dir = "logs\\" + "norm" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S" + "diff_convNum_dropout_0_larger_filters_3_hidden_256_neurons")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [27]:
history3 = modelCNN3.fit(
    train_generator,
    steps_per_epoch=50,
    validation_steps=1,
    epochs=150,
    validation_data=validation_generator,
    callbacks=[tensorboard_callback]
)

Epoch 1/150
50/50 [==============================] - 45s 667ms/step - loss: 5.2913 - accuracy: 0.0055 - val_loss: 5.2610 - val_accuracy: 0.0078
Epoch 2/150
50/50 [==============================] - 37s 739ms/step - loss: 5.1760 - accuracy: 0.0107 - val_loss: 5.0364 - val_accuracy: 0.0156
Epoch 3/150
50/50 [==============================] - 36s 707ms/step - loss: 4.8795 - accuracy: 0.0195 - val_loss: 4.7736 - val_accuracy: 0.0195
Epoch 4/150
50/50 [==============================] - 33s 656ms/step - loss: 4.5828 - accuracy: 0.0373 - val_loss: 4.3879 - val_accuracy: 0.0703
Epoch 5/150
50/50 [==============================] - 35s 706ms/step - loss: 4.2294 - accuracy: 0.0680 - val_loss: 3.8897 - val_accuracy: 0.1250
Epoch 6/150
50/50 [==============================] - 35s 696ms/step - loss: 3.9313 - accuracy: 0.0950 - val_loss: 3.7129 - val_accuracy: 0.0898
Epoch 7/150
50/50 [==============================] - 34s 684ms/step - loss: 3.7376 - accuracy: 0.1195 - val_loss: 3.6786 - val_accuracy:

In [29]:
y_pred3 = modelCNN3.predict(x_test)
print(classification_report(y_test.argmax(axis=1), y_pred3.argmax(axis=1)))

1563/1563 [==============================] - 47s 30ms/step
              precision    recall  f1-score   support

           0       0.62      0.76      0.68       246
           1       0.78      0.71      0.74       258
           2       0.56      0.29      0.38       254
           3       0.95      0.92      0.93       249
           4       0.75      0.81      0.78       247
           5       0.59      0.56      0.57       262
           6       0.74      0.53      0.62       255
           7       0.52      0.45      0.49       256
           8       0.91      0.75      0.82       246
           9       0.85      0.67      0.75       248
          10       0.66      0.94      0.78       266
          11       0.59      0.50      0.54       231
          12       0.52      0.66      0.58       240
          13       0.82      0.83      0.82       261
          14       0.94      0.91      0.93       239
          15       0.91      0.80      0.85       253
          16       0.8

In [30]:
modelCNN3.save(r'C:\Users\monop\Desktop\BOOTCAMP\projekt_DL\model068acc.h5')
